In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV, validation_curve
from sklearn.metrics import roc_auc_score, f1_score, plot_confusion_matrix, \
    plot_roc_curve, plot_precision_recall_curve, classification_report, precision_score, recall_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_sample_weight
from eli5.sklearn import PermutationImportance
import eli5
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import pickle
import warnings

warnings.filterwarnings("ignore")

### Features

- enrolle_id - Уникальный идентификатор кандидата
- city - Код города
- city_ development _index - Индекс развития города (масштабированный)
- gender - пол кандидата
- relevent_experience - релевантный опыт кандидата
- enrolled_university - тип обучения в университете (если был)
- education_level - уровень образования кандидата
- major_discipline - основная специальность по образованию
- experience -  общий стаж кандидата в годах
- company_size - размер компании в работниках
- company_type - тип работодателя
- lastnewjob - дельта в годах между предыдущей и текущей работах
- training_hours - кол-во завершенных часов обучения
- target 0 – Не ищу новую работу
- target 1 – Ищу новую работу 

### Inspiration

Спрогнозируйте вероятность того, что кандидат будет работать на компанию&
Features importance

In [ ]:
df = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
df_test = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')
sub = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/sample_submission.csv')

In [ ]:
df[:5]

In [ ]:
df_test[:5]

In [ ]:
sub[:5]

In [ ]:
set(df_test.enrollee_id)&set(set(df.enrollee_id))

# 1.Exploratory data analysis

In [ ]:
ax = sns.countplot(y="target", data=df, alpha=0.8)
total = df.shape[0]

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width() / total)
    x = p.get_x() + p.get_width()
    y = p.get_y() + p.get_height() / 2
    ax.annotate(percentage, (x, y))

plt.show()

Имеется небольшой дисбаланс классов, попробуем его решить без применения методик по устранению дисбаланса и с ними

In [ ]:
df.dtypes

Перевдем id города в тип object

Большинство признаков типа объект

In [ ]:
df.isna().mean() * 100

Максимальный процент пропуска в данных около 32%, это говорит о том, что  фичи с пропусками мы можем не выкидывать, так как могут нести потенциально часть информации для прогноза

In [ ]:
#выбираем признаки типа объект
cols_obj = list(df.dtypes[df.dtypes == object].index[1:])

#строим графики признаков типа объект от значения таргета, отнормируем на размер датасета
plt.figure(figsize=[15, 20])

i = 1
y, hue = "proportion", "target"

for f in cols_obj:
    plt.subplot(5, 2, i)
    df[[f, 'target']]\
        .value_counts(normalize=True)\
        .rename(y)\
        .reset_index()\
        .pipe((sns.barplot, "data"), x=f, y='proportion', hue='target', alpha=0.8)
    plt.title("Proportion of {}  by target".format(f))
    i += 1
plt.tight_layout()
plt.show()

Больше всего подержены поиску новой работы с учетом нормализации:
- мужчины
- студенты
- специализация STEM, возможно у людей больше выбора
- текущая компания типа PVT LTD (Private limited company, Н: НАО)
- имеющие за пречами 1 компанию в качестве опыта

In [ ]:
sns.boxplot(x='target', y='city_development_index', data=df)
plt.show()

In [ ]:
sns.displot(x='city_development_index',
            hue='target',
            data=df,
            stat="probability")
plt.show()

Вероятность поиска новой работы немного выше, чем ниже индекс разития города

In [ ]:
sns.boxplot(x='target', y='training_hours', data=df)
plt.show()

In [ ]:
sns.displot(x='training_hours',
            hue='target',
            data=df,
            stat="probability")
plt.show()

По количеству завершенных часов обучения нет никаких нагляднных инсайтов

In [ ]:
plt.figure(figsize=[10, 5])
sns.boxplot(x='company_size', y='city_development_index', data=df)
plt.show()

У больших компаний инедекс развития в среднем выше

In [ ]:
plt.figure(figsize=[10, 5])
sns.boxplot(x='major_discipline', y='city_development_index', data=df)
plt.show()

Часть из дисциплин присуща только развитым городам, люди с образованием STEM  больше живут в городах с низким индексом развития, плюс до этого выяснили, что чем ниже индекс, тем немного больше людей предрасположено к поиску новой работы, есть некоторая взаимосвязь индекса, образования и поиска работы

## 1.2 Feature Engineering

Есть гипотеза (из прошлого анализа), что к поиску новой работы склонны молодые студенты, но мы не имеем возраста человека в датасете, но можем его потенциально обозначить, используя несколько признаков

In [ ]:
df_for_age = df[['enrollee_id', 'education_level', 'experience', 'last_new_job']]\
    .groupby(['education_level', 'experience', 'last_new_job']).sum('enrollee_id').reset_index()

In [ ]:
df_for_age[:5]

In [ ]:
plt.figure(figsize=[15, 5])

df[['experience', 'last_new_job']]\
    .value_counts(normalize=True)\
    .rename(y)\
    .reset_index()\
    .pipe((sns.barplot, "data"), x='experience', y='proportion', hue='last_new_job', alpha=0.8);

Есть некая заимосвязь между опытом работы и числом компаний

In [ ]:
plt.figure(figsize=[15, 5])

df[['experience', 'gender']]\
    .value_counts(normalize=True)\
    .rename(y)\
    .reset_index()\
    .pipe((sns.barplot, "data"), x='experience', y='proportion', hue='gender', alpha=0.8);

In [ ]:
plt.figure(figsize=[15, 5])

df[['education_level', 'experience']]\
    .value_counts(normalize=True)\
    .rename(y)\
    .reset_index()\
    .pipe((sns.barplot, "data"), x='experience', y='proportion', hue='education_level', alpha=0.8);

Явно выделяются люди с  Phd с большим опытом, но для выяления возраста фича education_level не очень применима кроме исключения Phd для молодых специалистов

Возьмем примерные временные рамки для возрастных категорий, если данных будет больше, можно сделать это более точным, чем ориентация на образование и опыт

- young < 20
- adult - 20-40
- middle - 40-60
- old - >60

In [ ]:
def get_age_category(x, y):
    if ((x in ['1', '2', '3', '4', '5', '6', '<1', '7', '8']) &
            ((y in ['Phd']) | (y in ['Masters']) | (y in ['Graduate']))):
        return 'Adult'
    elif ((x in ['1', '2', '3', '<1']) & (y != 'Phd') & (y != 'Masters') &
          (y != 'Graduate')):
        return 'Young'
    elif (x in ['9', '10', '11', '12', '13', '14', '15']):
        return 'Middle'
    elif (x in ['16', '17', '18', '19', '20', '>20']):
        return 'Old'

In [ ]:
df['Age'] = df.apply(lambda row: get_age_category(row['experience'], row['education_level']), axis=1)

In [ ]:
df.Age.value_counts()

In [ ]:
df[:5]

Подобные дейтсивия можно совершить и с другими гипотезами, чтобы добавить новых фичей в исходную дату, а также лучше понять сами данные, какую модель использовать

## 1.2 Fill empty

In [ ]:
df.isna().mean()[df.isna().mean() > 0]

Посмотрим какие уже есть значения, которыми можем заменить пропуски

In [ ]:
for i in df.isna().mean()[df.isna().mean() > 0].index:
    print(i, df[i].unique(), '\n')

Лучшим выходом будет заменить пропуски как 'None'

In [ ]:
for i in df.isna().mean()[df.isna().mean() > 0].index:
    df[i] = df[i].fillna('None')

# 2. Modeling

## 2.1 Logistic Regression

Мы имеем множество категориальных данных, также можно отметить, что данные возможно имеют нелинейные зависимости, поэтому лучше не использовать линейные модели, они хорошо работают, когда классы линейно разделимы, но в качесте эксперимена, возьмем частный случай линейной модели, основаный на вероятностой модели

In [ ]:
X = pd.get_dummies(df, columns=df.dtypes[df.dtypes == object].index).drop(
    ['enrollee_id', 'target'], axis=1)
y = df.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=10)

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_pred_proba = lr.predict_proba(X_test)[:, 1]
y_pred_lr = lr.predict(X_test)

Так как классы не сбалансированы, смотреть на roc-auc не имеет смысла, он перетягивает на себя скор наибольшего класса, поэтому данная метрика на первый взгляд может давать хороший результат, но если взглянет на pr-rec, но увидим, что недостающий класс плохо определяется

In [ ]:
print(classification_report(y_test, y_pred_lr))

In [ ]:
df_report = pd.DataFrame(columns={'ROC-AUC'}, data=[0])

df_report['ROC-AUC'] = roc_auc_score(y_test, y_pred_proba)
df_report['F1'] = f1_score(y_test, y_pred_lr)
df_report['precision_0'] = precision_score(y_test, y_pred_lr, pos_label=0)
df_report['recall_0'] = recall_score(y_test, y_pred_lr, pos_label=0)
df_report['precision_1'] = precision_score(y_test, y_pred_lr, pos_label=1)
df_report['recall_1'] = recall_score(y_test, y_pred_lr, pos_label=1)

df_report.index = ['LogisticRegression']

In [ ]:
df_report

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(lr, X_test, y_test, ax=ax1)
plot_roc_curve(lr, X_test, y_test, ax=ax2);

PR-REC AUC близка к 0.5, также f1=0.37, что говорит о том, что алгоритм пока не способен как либо различать классы, больше всего он присваивает объектам класс 0, попробуем подобрать наилучшие параметры, а также сделаем стратиикацию, которая часто применяется при дисбалансе классов

In [ ]:
# Grid Search
lr_skf = LogisticRegression(class_weight='balanced')
skf = StratifiedShuffleSplit(n_splits=5, random_state=10)

param = {'C': np.linspace(0.001, 10, 10), 'penalty': ['l1', 'l2']}

# refit - пол умолчаю, то есть при предикте уже используем лучшую модель
clf_lr = GridSearchCV(
    lr_skf, param, scoring='roc_auc', refit=True, cv=skf)
clf_lr.fit(X_train, y_train)

print('Best roc_auc: {:.4}, with best C: {}'.format(
    clf_lr.best_score_, clf_lr.best_params_))

In [ ]:
# Чтобы посмотреть все доступные метрики
sklearn.metrics.SCORERS.keys()

In [ ]:
# Функция для возвращения таблицы с метриками
def get_scores(report_df, model, X_test, y_test, name):

    report = pd.DataFrame(columns={'ROC-AUC'}, data=[0])
    report['ROC-AUC'] = roc_auc_score(y_test,
                                      model.predict_proba(X_test)[:, 1])
    report['F1'] = f1_score(y_test, model.predict(X_test))
    report['precision_0'] = precision_score(
        y_test, model.predict(X_test), pos_label=0)
    report['precision_1'] = precision_score(
        y_test, model.predict(X_test), pos_label=1)
    report['recall_0'] = recall_score(
        y_test, model.predict(X_test), pos_label=0)
    report['recall_1'] = recall_score(
        y_test, model.predict(X_test), pos_label=1)

    report.index = [name]
    report_df = report_df.append(report)
    return report_df

In [ ]:
df_report = get_scores(df_report, clf_lr, X_test,
                       y_test, 'LogisticRegression_skf')

In [ ]:
df_report

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(clf_lr, X_test, y_test, ax=ax1)
plot_roc_curve(clf_lr, X_test, y_test, ax=ax2);

Заметно подросла полнота и в связи с этим f1, но особой разницы при балансировке классов не видно с учетом поиска лучших параметров и стратифицированной выборки для обучения

Попробуем добавить объектов недостающего класса

In [ ]:
oversample = RandomOverSampler(sampling_strategy=1.0)
X_over, y_over = oversample.fit_resample(X_train, y_train)

In [ ]:
# Grid Search
lr_skf = LogisticRegression()
skf = StratifiedShuffleSplit(n_splits=5, random_state=10)

param = {'C': np.linspace(0.001, 10, 10), 'penalty': ['l1', 'l2']}

# verbose - печать резльутата выполнения обучения, чем больше  значение (по умол 0), тем больше информации
clf_lr_over = GridSearchCV(lr_skf, param, scoring='roc_auc', cv=skf, verbose=1)
clf_lr_over.fit(X_over, y_over)

print('Best roc_auc: {:.4}, with best C: {}'.format(
    clf_lr_over.best_score_, clf_lr_over.best_params_))

In [ ]:
df_report = get_scores(df_report, clf_lr_over, X_test,
                       y_test, 'LogisticRegression_skf_imb')

In [ ]:
df_report

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(clf_lr_over, X_test, y_test, ax=ax1)
plot_roc_curve(clf_lr_over, X_test, y_test, ax=ax2);

Заметим, что очень хорошо работает встроенный параметр class_weight='balanced', но для ваших данных лучше тестировать оба способа, чтобы посмотреть какой отработает лучше, и в целом LogisticRegression хорошо применима в случае линнейной зависимости между целевой переменной и признаками, в более сложных случах хорошо подходят композиции из решающих деревьев, так как сами решающие деревья легко переобучаются + затраты на произодительность 

## 2.2 Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

df_report = get_scores(df_report, rf, X_test, y_test, 'RandomForestClassifier')

In [ ]:
df_report

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(rf, X_test,y_test, ax=ax1)
plot_roc_curve(rf, X_test,y_test, ax=ax2);

Видим явное увеличение метрик базового RandomForest по сравнению с базовым LogisticRegression, о чем и упоминали ранее при подборе алгоритма

In [ ]:
# Grid Search
rf_skf = RandomForestClassifier(class_weight='balanced')
skf = StratifiedShuffleSplit(n_splits=5, random_state=10)

param = {'bootstrap': [True],
         'max_depth': [10, 30],
         'n_estimators': [600, 1000]}

clf_rf = GridSearchCV(
    rf_skf, param, scoring='roc_auc', refit=True, cv=skf, verbose=3, n_jobs=-1)
clf_rf.fit(X_train, y_train)

print('Best roc_auc: {:.4}, with best C: {}'.format(
    clf_rf.best_score_, clf_rf.best_params_))

In [ ]:
df_report = get_scores(df_report, clf_rf, X_test, y_test, 'RandomForestClassifier_skf')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(clf_rf, X_test, y_test, ax=ax1)
plot_roc_curve(clf_rf, X_test, y_test, ax=ax2);

In [ ]:
df_report

При поиске наилучших параметров, мы улучшили практически все метрики, кроме точности особенно на 1 классе, причем recall увеличился, здесь уже нужно отталкиваться от бизнес-задачи, что важнее в данном случае 

In [ ]:
# Grid Search
rf_skf = RandomForestClassifier()
skf = StratifiedShuffleSplit(n_splits=5, random_state=10)

param = {'bootstrap': [True],
         'max_depth': [10, 30],
         'n_estimators': [200, 600]}

clf_rf_over = GridSearchCV(
    rf_skf, param, scoring='roc_auc', refit=True, cv=skf, verbose=3, n_jobs=-1)
clf_rf_over.fit(X_over, y_over)

print('Best roc_auc: {:.4}, with best C: {}'.format(
    clf_rf_over.best_score_, clf_rf_over.best_params_))

In [ ]:
df_report = get_scores(df_report, clf_rf_over, X_test, y_test,
                       'RandomForestClassifier_skf_imb')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(clf_rf_over, X_test, y_test, ax=ax1)
plot_roc_curve(clf_rf_over, X_test, y_test, ax=ax2);

In [ ]:
df_report

На сэмплированных данных точность на 1 классе стала чуть выше, но мы заплатили точность 0 класса, опять же нужно отталкиваться от бизнеса, например в данном примере, нам важно знать, будет ли кандидат искать новую работу, то что он не ищет ее, нас не так интересует, поэтому выбираем RandomForestClassifier_skf по f1

## 2.3 Lightgbm 

Бустинг хорошо тем, что обучается прежде всего учится на ошибках построенных последовательно алгоритмов, плюс конечно он отрабатывает быстрее, чем случайные леса

Данный алгоритм отрабатывает еще быстрее чем xgboost, а также умеет работать с категориальными переменными

In [ ]:
X_ = df.drop(['enrollee_id', 'target'], axis=1)
y_ = df['target']

for c in X_.columns:
    col_type = X_[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X_[c] = X_[c].astype('category')

In [ ]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(
    X_, y_, test_size=0.25, random_state=10)

In [ ]:
lgb = LGBMClassifier()
lgb.fit(X_train_, y_train_, categorical_feature='auto')

df_report = get_scores(df_report, lgb, X_test_,
                       y_test_, 'LGBMClassifier')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(lgb, X_test_, y_test_, ax=ax1)
plot_roc_curve(lgb, X_test_, y_test_, ax=ax2);

In [ ]:
df_report

In [ ]:
# Grid Search
lgb_skf = LGBMClassifier(class_weight='balanced', categorical_feature='auto')
skf = StratifiedShuffleSplit(n_splits=5, random_state=10)

param = {'learning_rate': [0.005, 0.1],
         'num_leaves': [30, 50],
         'n_estimators': [500, 600]}

clf_lgb = GridSearchCV(lgb_skf, param, scoring='roc_auc',
                       cv=skf, verbose=3, n_jobs=-1)
clf_lgb.fit(X_train_, y_train_)

print('Best roc_auc: {:.4}, with best C: {}'.format(
    clf_lgb.best_score_, clf_lgb.best_params_))

In [ ]:
df_report = get_scores(df_report, clf_lgb, X_test_, y_test_,
                       'LGBMClassifier_skf')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(clf_lgb, X_test_, y_test_, ax=ax1)
plot_roc_curve(clf_lgb, X_test_, y_test_, ax=ax2);

In [ ]:
df_report

In [ ]:
X_over_, y_over_ = oversample.fit_resample(X_train_, y_train_)

In [ ]:
# Grid Search
lgb_skf = LGBMClassifier(categorical_feature='auto')
skf = StratifiedShuffleSplit(n_splits=5, random_state=10)

param = {'learning_rate': [0.005, 0.1],
         'num_leaves': [30, 50],
         'n_estimators': [500, 600]}

clf_lgb_over = GridSearchCV(lgb_skf, param, scoring='roc_auc',
                            cv=skf, verbose=3, n_jobs=-1)
clf_lgb_over.fit(X_over_, y_over_)

print('Best roc_auc: {:.4}, with best C: {}'.format(
    clf_lgb_over.best_score_, clf_lgb_over.best_params_))

In [ ]:
df_report = get_scores(df_report, clf_lgb_over, X_test_, y_test_,
                       'LGBMClassifier_skf_imb')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(clf_lgb_over, X_test_, y_test_, ax=ax1)
plot_roc_curve(clf_lgb_over, X_test_, y_test_, ax=ax2);

In [ ]:
df_report

Увеличение объектов недостающего класса негативно сказалось на LGBMClassifier

## 2.4 Stacking

Попробуем применить стекинг над моделями, объединим разные модели, у которых f1 более 60%

Посмотрим, можем ли мы улучшить производительность, комбинируя предсказания моделей с помощью "soft voting". 

**Soft voting** - это argmax суммы предсказанных вероятностей.

In [ ]:
lgbm_ = LGBMClassifier(learning_rate=0.005, n_estimators=600, num_leaves=50)
rf_ = RandomForestClassifier(bootstrap=True, max_depth=10, n_estimators=1000)
lr_ = LogisticRegression(C=4.445, penalty= 'l2')

clf_st = VotingClassifier(estimators=[
    ('lr', lr_), ('rf', rf_), ('lgb', lgbm_)], voting='soft')

clf_st.fit(X_over.values, y_over.values)

In [ ]:
df_report = get_scores(df_report, clf_st, X_test, y_test,
                       'VotingClassifier')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(clf_st, X_test, y_test, ax=ax1)
plot_roc_curve(clf_st, X_test, y_test, ax=ax2);

In [ ]:
df_report

При помощи VotingClassifier мы добились наилучшего результата, в особенности на 1 классе, в качестве конечного решения можно выбрать последнюю модель

## 3. Permutation feature importance

Измерение важности признако за счет перестановок внутри одной фичи (например, строки): если после перестановки ошибка модели увеличилась, то фича "важна", если ошибка произошла без изменений, то "неважна".

In [ ]:
perm = PermutationImportance(clf_st, scoring='f1').fit(X_test, y_test)

In [ ]:
eli5.show_weights(perm, feature_names=X_test.columns.tolist())

In [ ]:
perm_importance = eli5.explain_weights_df(perm).sort_values(by='weight',
                                                            ascending=False)
perm_importance = perm_importance[perm_importance['weight'] > 0]
perm_importance['f'] = perm_importance['feature'].apply(lambda x: int(x[1:]))
cols_perm = list(X_test.columns[perm_importance['f']])
perm_importance['feature'] = cols_perm

In [ ]:
perm_importance[:10]

Ранее выдвинутая гипотеза о влиянии индекса города на поиск работы, подтвердилась при поиске важный фичей, также важными являются это отсутствие информации о текущей компании, а также что человек не работал до этого, под это описание отлично подходит выпускник ВУЗа

## 4. Predict

Пример для предикта, когда требуется дать ответ для соревнования

In [ ]:
df_test['Age'] = df_test.apply(lambda row: get_age_category(
    row['experience'], row['education_level']), axis=1)

X_end = pd.get_dummies(df_test, columns=df_test.dtypes[df_test.dtypes == object].index).drop(
    ['enrollee_id'], axis=1)

In [ ]:
X_end[list(set(X_end.columns) ^ set(X_train.columns))] = 0

X_end = X_end[X_train.columns]
sub['target'] = clf_st.predict(X_end)

In [ ]:
sub

In [ ]:
sub.to_csv('sample_submission.csv')